In [1]:
!pip install earthengine-api
import ee
from google.colab import auth
auth.authenticate_user()
ee.Authenticate()
ee.Initialize()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=nDjrbQlsOnECWdErjyQLldTL-GwaVXGZ6ieqIjiefW0&tc=3cuyY5AsuflIByR7Ql1Ycf2YWIQDLrqVz4l-wR_idOI&cc=oXHAfR6qJXzy0V8kQn4Bp4EeilAPeHzeoiKWPaJ6JnA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk6JIGhQtsbuq9XBFxNnaam-lduLppYFJt4rxSivrZA8eZZi0etA_0g

Successfully saved authorization token.


In [11]:
# Define the polygon coordinates
coordinates = [
    [
        -4.824638002703523,
        57.04707818690099
    ],
    [
        -4.841187119373046,
        57.10585217843848
    ],
    [
        -5.05745913722177,
        57.09366145918008
    ],
    [
        -5.031842011417524,
        57.022407291450776
    ],
    [
        -4.824638002703523,
        57.04707818690099
    ]
]

# Create an ee.Geometry.Polygon object
polygon = ee.Geometry.Polygon([coordinates])

In [15]:
import datetime
import folium

# Define location coordinates
#longitude = -4.870611854790898
#latitude = 57.01166898422638
#point = ee.Geometry.Point([longitude, latitude])

# Define time range
start_year = 2015
end_year = 2022

# Import the MODIS MOD17A2H dataset (Gross Primary Productivity, 8-day composite)
gpp_dataset = ee.ImageCollection('MODIS/006/MOD17A2H')

# Import the Landsat 8 dataset (optical imagery)
landsat8_dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Function to add Earth Engine layers to the map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Initialize the map
my_map = folium.Map(location=[57.095549291293935, -4.939520516668836], zoom_start=10)

# Loop through the years and calculate the yearly GPP
for year in range(start_year, end_year + 1):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'

    # Calculate the mean GPP for the year
    annual_gpp = gpp_dataset.filterDate(start_date, end_date) \
        .filterBounds(polygon) \
        .select('Gpp') \
        .sum() \
        .multiply(0.0001) \
        .multiply(ee.Image.pixelArea()) \
        .reduceRegion(reducer=ee.Reducer.sum(), geometry=polygon, scale=500)

    
    # Retrieve the GPP value
    forest_carbon_absorbed = annual_gpp.getInfo()['Gpp']

    # Convert grams of carbon to kilograms
    forest_carbon_absorbed_kg = forest_carbon_absorbed / 1000

    print(f"Forest carbon absorbed (in kg of carbon) in {year}: {forest_carbon_absorbed_kg}")

    # Display the Landsat 8 imagery for the year
    landsat8_image = landsat8_dataset.filterDate(start_date, end_date) \
        .filterBounds(polygon) \
        .median() \
        .clip(polygon)

    vis_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max':3000,
        'gamma': 1.4
    }
    my_map.add_ee_layer(landsat8_image, vis_params, f'Landsat 8 - {year}')

     # Add the MODIS GPP layer for the year
    modis_gpp_image = gpp_dataset.filterDate(start_date, end_date) \
        .filterBounds(polygon) \
        .select('Gpp') \
        .sum() \
        .clip(polygon)

    modis_gpp_vis_params = {
        'min': 0,
        'max': 2000,
        'palette': ['red', 'yellow', 'green']
    }

    my_map.add_ee_layer(modis_gpp_image, modis_gpp_vis_params, f'MODIS GPP - {year}')

# Add the layer control and display the map
folium.LayerControl().add_to(my_map)
my_map

Forest carbon absorbed (in kg of carbon) in 2015: 51427.42568217601
Forest carbon absorbed (in kg of carbon) in 2016: 51081.69656076536
Forest carbon absorbed (in kg of carbon) in 2017: 44415.76779240904
Forest carbon absorbed (in kg of carbon) in 2018: 57086.92659012047
Forest carbon absorbed (in kg of carbon) in 2019: 50548.75874144951
Forest carbon absorbed (in kg of carbon) in 2020: 51782.2168823581
Forest carbon absorbed (in kg of carbon) in 2021: 57059.84072984682
Forest carbon absorbed (in kg of carbon) in 2022: 57127.40611124361
